In [4]:
#Step to ensure that the venv is being used for the project not local copies, should point at .venv in project.
import sys, shutil
print("python:", sys.executable)
print("uv:", shutil.which("uv")) 

python: c:\WorkSpace\Hackathon\Hackathon-2025\src\agent\attractions\.venv\Scripts\python.exe
uv: c:\WorkSpace\Hackathon\Hackathon-2025\src\agent\attractions\.venv\Scripts\uv.EXE


In [5]:
# installs into the current Jupyter kernel environment
%pip install -U uv 
#! to run shell commands
!uv pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Audited 13 packages in 34ms


In [6]:
# LangChain + MCP Setup for Attractions Booking (HTTP-based for Jupyter)
import os
from dotenv import load_dotenv
import asyncio
import json
import requests
from typing import Dict, Any, List

# Updated LangChain imports for 0.3.x
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Official MCP adapter imports for HTTP transport
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# Load environment variables
load_dotenv()

print("✅ Updated imports with official MCP adapter loaded successfully!")


✅ Updated imports with official MCP adapter loaded successfully!


In [7]:
# MCP Client Setup using Official Adapter with HTTP Transport
import subprocess
import time

# Global MCP client for HTTP
mcp_client = None

async def create_mcp_tools():
    """Create MCP tools using the official LangChain MCP adapter with HTTP transport"""
    global mcp_client
    
    try:
        # Create MultiServerMCPClient with streamable_http transport
        mcp_client = MultiServerMCPClient({
            "attractions": {
                "transport": "streamable_http",
                "url": "http://localhost:8008/mcp/"
            }
        })
        
        # Get tools from the MCP server
        tools = await mcp_client.get_tools()
        print(f"📋 Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
        return tools
        
    except Exception as e:
        print(f"Error connecting to MCP HTTP server: {e}")
        return []

print("🔗 MCP HTTP adapter setup ready!")


🔗 MCP HTTP adapter setup ready!


In [8]:
# Load MCP Tools using Official Adapter
# The tools will be loaded dynamically when setting up the agent
# No need to manually create tool wrappers - the adapter handles this automatically
print("🛠️ Ready to load MCP tools via official adapter!")


🛠️ Ready to load MCP tools via official adapter!


In [9]:
async def setup_agent():
    """Setup LangChain agent with MCP tools using official adapter"""
    
    # Initialize LLM for Azure OpenAI
    # can get this from Azure Open Ai service -> Azure Ai Foundary Portal
    from langchain_openai import AzureChatOpenAI
    
    llm = AzureChatOpenAI(
        deployment_name="gpt-5-chat",  # Your Azure deployment name
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_version="2025-01-01-preview", 
        temperature=0.1
    )
    
    # Load MCP tools using official adapter
    tools = await create_mcp_tools()
    
    if not tools:
        print("⚠️ No MCP tools loaded. Make sure the MCP server is accessible.")
        return None
    
    print(f"📋 Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
    
    # Create system prompt
    system_prompt = """You are a helpful travel assistant that can help users find and book attractions.
    
    You have access to multiple MCP tools for tourist attractions, including:
    - Searching for attractions by location and category
    - Getting detailed attraction information
    - Booking attractions for visitors
    - Getting random attraction suggestions
    - And more...
    
    When users ask about travel plans, use these tools to provide comprehensive information.
    Always be helpful and provide practical advice."""
    
    # Create prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create agent
    agent = create_tool_calling_agent(llm, tools, prompt)
    
    # Create agent executor
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
    return agent_executor

# Initialize the agent (now async)
agent_executor = None
print("🤖 Agent setup function ready! Run the next cell to initialize.")


🤖 Agent setup function ready! Run the next cell to initialize.


In [18]:
# Initialize the agent with MCP tools
async def initialize_agent():
    """Initialize the agent with MCP tools"""
    global agent_executor
    print("🔄 Initializing agent with MCP tools...")
    agent_executor = await setup_agent()
    if agent_executor:
        print("🤖 LangChain agent with MCP tools ready!")
    else:
        print("❌ Failed to initialize agent. Check MCP server connection.")

# Run the initialization
await initialize_agent()


🔄 Initializing agent with MCP tools...
📋 Loaded 5 MCP tools: ['get_attraction_details', 'search_attractions', 'get_random_attraction', 'book_attraction', 'search_and_format_attractions']
📋 Loaded 5 MCP tools: ['get_attraction_details', 'search_attractions', 'get_random_attraction', 'book_attraction', 'search_and_format_attractions']
🤖 LangChain agent with MCP tools ready!
📋 Loaded 5 MCP tools: ['get_attraction_details', 'search_attractions', 'get_random_attraction', 'book_attraction', 'search_and_format_attractions']
📋 Loaded 5 MCP tools: ['get_attraction_details', 'search_attractions', 'get_random_attraction', 'book_attraction', 'search_and_format_attractions']
🤖 LangChain agent with MCP tools ready!


In [19]:
# User Input Handler
async def process_user_input(user_input: str) -> str:
    """Process user input and return LLM response using MCP tools"""
    if not agent_executor:
        return "❌ Agent not initialized. Please run the initialization cell first."
    
    try:
        # Use the agent to process the input
        result = await agent_executor.ainvoke({"input": user_input})
        return result["output"]
    except Exception as e:
        return f"Error processing request: {str(e)}"

# Interactive function for easy testing
async def ask_assistant(question: str):
    """Easy-to-use function for asking the travel assistant"""
    print(f"🧳 User: {question}")
    print("🤖 Assistant: ", end="")
    
    # Await the async function directly (do not use asyncio.run() in notebooks)
    response = await process_user_input(question)
    print(response)
    return response

print("💬 User input handler ready!")

💬 User input handler ready!


In [20]:
# Test MCP server connectivity and tools
async def test_mcp_connection():
    """Test MCP server connection and list available tools"""
    tools = await create_mcp_tools()
    if tools:
        print(f"✅ MCP HTTP server connected successfully!")
        print(f"📋 Available tools: {[tool.name for tool in tools]}")
        for tool in tools:
            print(f"  - {tool.name}: {tool.description}")
    else:
        print("❌ Failed to connect to MCP HTTP server")

# Test MCP HTTP connection
await test_mcp_connection()


📋 Loaded 5 MCP tools: ['get_attraction_details', 'search_attractions', 'get_random_attraction', 'book_attraction', 'search_and_format_attractions']
✅ MCP HTTP server connected successfully!
📋 Available tools: ['get_attraction_details', 'search_attractions', 'get_random_attraction', 'book_attraction', 'search_and_format_attractions']
  - get_attraction_details: Get detailed information about a specific tourist attraction

Args:
    attraction_id: Unique ID of the attraction
    
Returns:
    AttractionDetails object as dictionary with attraction info, facilities, and visiting tips

  - search_attractions: Search for tourist attractions with optional filters

Args:
    location: Location to search in (e.g., "Paris", "India", "Italy")
    category: Category filter - "historical", "natural", "cultural", "religious", "modern", "museums", "parks", "beaches", "mountains", "architecture", "entertainment", "adventure"
    limit: Maximum number of results (1-100, default: 20)
    
Returns:
    A

In [13]:
# Cleanup function for HTTP MCP client
async def cleanup_mcp():
    """Cleanup MCP client and server resources"""
    global mcp_client
    if mcp_client:
        try:
            await mcp_client.close()
            print("✅ MCP client closed")
        except Exception as e:
            print(f"Warning: Error closing MCP client: {e}")
        mcp_client = None

print("🧹 Cleanup function ready!")


🧹 Cleanup function ready!


In [22]:
# 🚀 EXAMPLE USAGE - Run this cell after setting up your API key!

# Example 1: Simple question
await ask_assistant("What are some popular attractions in Paris?")

print("\n" + "="*50 + "\n")

# Example 2: Weather + attractions
await ask_assistant("I'm planning to visit Tokyo tomorrow. What's the weather like and what attractions should I visit?")

print("\n" + "="*50 + "\n")

# Example 3: Interactive input (uncomment to use)
# user_question = input("Ask me anything about travel: ")
# await ask_assistant(user_question)

🧳 User: What are some popular attractions in Paris?
🤖 Assistant: 

> Entering new AgentExecutor chain...

Invoking: `search_and_format_attractions` with `{'location': 'Paris', 'limit': 10}`



Invoking: `search_and_format_attractions` with `{'location': 'Paris', 'limit': 10}`


🎯 Found 2 attractions in Paris:

1. 🏛️ **Eiffel Tower**
   📍 Paris, France
   🏷️ Architecture
   ⭐ 4.6/5.0
   💰 €29.40 - €73.30

2. 🏛️ **Louvre Museum**
   📍 Paris, France
   🏷️ Museums
   ⭐ 4.4/5.0
   💰 €17

🎯 Found 2 attractions in Paris:

1. 🏛️ **Eiffel Tower**
   📍 Paris, France
   🏷️ Architecture
   ⭐ 4.6/5.0
   💰 €29.40 - €73.30

2. 🏛️ **Louvre Museum**
   📍 Paris, France
   🏷️ Museums
   ⭐ 4.4/5.0
   💰 €17

Here are some popular attractions in Paris:

1. **Eiffel Tower** 🗼  
   - **Category:** Architecture  
   - **Rating:** ⭐ 4.6/5.0  
   - **Price:** €29.40 - €73.30  

2. **Louvre Museum** 🎨  
   - **Category:** Museums  
   - **Rating:** ⭐ 4.4/5.0  
   - **Price:** €17  

Would you like me to share mor